## Introduction

In this tutorial, you learn how to use Google Cloud AI tools to quickly bring the power of Large Language Models to enterprise systems.  

This tutorial covers the following -

*   What are embeddings - what business challenges do they help solve ?
*   Understanding Text with Vertex AI Text Embeddings
*   Find Embeddings fast with Vertex AI Vector Search
*   Grounding LLM outputs with Vector Search

This tutorial is based on [the blog post](https://cloud.google.com/blog/products/ai-machine-learning/how-to-use-grounding-for-your-llms-with-text-embeddings), combined with sample code.


### Prerequisites

This tutorial is designed for developers who has basic knowledge and experience with Python programming and machine learning.

If you are not reading this tutorial in Qwiklab, then you need to have a Google Cloud project that is linked to a billing account to run this. Please go through [this document](https://cloud.google.com/vertex-ai/docs/start/cloud-environment) to create a project and setup a billing account for it.

### Choose the runtime environment

The notebook can be run on either Google Colab or [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

- To use Colab: Click [this link](https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.ipynb) to open the tutorial in Colab.

- To use Workbench: If it is the first time to use Workbench in your Google Cloud project, open [the Workbench console](https://console.cloud.google.com/vertex-ai/workbench) and click ENABLE button to enable Notebooks API. Then click [this link](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/embeddings/intro-textemb-vectorsearch.ipynb),  and select an existing notebook or create a new notebook.


### How much will this cost?

In case you are using your own Cloud project, not a temporary project on Qwiklab, you need to spend roughly a few US dollars to finish this tutorial.

The pricing of the Cloud services we will use in this tutorial are avilable in the following pages:

- [Vertex AI Embeddings for Text](https://cloud.google.com/vertex-ai/pricing#generative_ai_models)
- [Vertex AI Vector Search](https://cloud.google.com/vertex-ai/pricing#matchingengine)
- [BigQuery](https://cloud.google.com/bigquery/pricing)
- [Cloud Storage](https://cloud.google.com/storage/pricing)
- [Vertex AI Workbench](https://cloud.google.com/vertex-ai/pricing#notebooks) if you use one

You can use the [Pricing Calculator](https://cloud.google.com/products/calculator) to generate a cost estimate based on your projected usage. The following is an example of rough cost estimation with the calculator, assuming you will go through this tutorial a couple of time.

<img src="https://storage.googleapis.com/github-repo/img/embeddings/vs-quickstart/pricing.png" width="50%"/>

### **Warning: delete your objects after the tutorial**

In case you are using your own Cloud project, please make sure to delete all the Indexes, Index Endpoints and Cloud Storage buckets (and the Workbench instance if you use one) after finishing this tutorial. Otherwise the remaining assets would incur unexpected costs.


# Bringing Gen AI and LLMs to production services

Many people are now starting to think about how to bring Gen AI and LLMs to production services, and facing with several challenges.

- "How to integrate LLMs or AI chatbots with existing IT systems, databases and business data?"
- "We have thousands of products. How can I let LLM memorize them all precisely?"
- "How to handle the hallucination issues in AI chatbots to build a reliable service?"

Here is a quick solution: **grounding** with **embeddings** and **vector search**.

What is grounding? What are embedding and vector search? In this tutorial, we will learn these crucial concepts to build reliable Gen AI services for enterprise use. But before we dive deeper, let's try the demo below.

# Vertex AI Embeddings for Text

With the [Vertex AI Embeddings for Text](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings), you can easily create a text embedding with LLM. The product is also available on [Vertex AI Model Garden](https://cloud.google.com/model-garden)

![](https://storage.googleapis.com/github-repo/img/embeddings/textemb-vs-notebook/7.png)

This API is designed to extract embeddings from texts. It can take text input up to 3,072 input tokens, and outputs 768 dimensional text embeddings.

# Text Embeddings in Action
## Setup

Before get started with the Vertex AI services, we need to setup the following.

* Install Python SDK
* Environment variables
* Authentication (Colab only)
* Enable APIs
* Set IAM permissions

### Install Python SDK

In [ ]:
# Install Vertex AI LLM SDK
! pip install --user --upgrade google-cloud-aiplatform==1.47.0 langchain==0.1.14 langchain-google-vertexai==0.1.3 typing_extensions==4.9.0

# Dependencies required by Unstructured PDF loader
! sudo apt -y -qq install tesseract-ocr libtesseract-dev
! sudo apt-get -y -qq install poppler-utils
! pip install --user --upgrade unstructured==0.12.4 pdf2image==1.17.0 pytesseract==0.3.10 pdfminer.six==20221105
! pip install --user --upgrade pillow-heif==0.15.0 opencv-python==4.9.0.80 unstructured-inference==0.7.24 pikepdf==8.13.0 pypdf==4.0.1

# For Matching Engine integration dependencies (default embeddings)
! pip install --user --upgrade tensorflow_hub==0.16.1 tensorflow_text==2.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-cloud-storage to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 121752 files and directories

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 13.5 MB/s eta 0:00:00


# Download custom Python modules and utilities
The cell below will download some helper functions needed for using Vertex AI Matching Engine in this notebook. These helper functions were created to keep this notebook more tidy and concise, and you can also view them directly on Github.

In [ ]:
import os
import urllib.request

if not os.path.exists("utils"):
    os.makedirs("utils")

url_prefix = "https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/language/use-cases/document-qa/utils"
files = ["__init__.py", "matching_engine.py", "matching_engine_utils.py"]

for fname in files:
    urllib.request.urlretrieve(f"{url_prefix}/{fname}", filename=f"utils/{fname}")

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
#Authenticating your notebook environment
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

# Enable APIs
Run the following to enable APIs for Compute Engine, Vertex AI, Cloud Storage and BigQuery with this Google Cloud project.

In [ ]:
! gcloud services enable compute.googleapis.com aiplatform.googleapis.com storage.googleapis.com bigquery.googleapis.com --project {PROJECT_ID}

Operation "operations/acat.p2-223914638453-ac2120aa-c7dd-4dcd-a98b-1df67b2a22a5" finished successfully.


### Set IAM permissions

Also, we need to add access permissions to the default service account for using those services.

- Go to [the IAM page](https://console.cloud.google.com/iam-admin/) in the Console
- Look for the principal for default compute service account. It should look like: `<project-number>-compute@developer.gserviceaccount.com`
- Click the edit button at right and click `ADD ANOTHER ROLE` to add `Vertex AI User`, `BigQuery User` and `Storage Admin` to the account.

This will look like this:

![](https://storage.googleapis.com/github-repo/img/embeddings/vs-quickstart/iam-setting.png)

# Environment variables

In [ ]:
# get project ID
PROJECT_ID = ! gcloud config get project

LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")
    # define project information


# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

# Import libraries

In [ ]:
import vertexai

#PROJECT_ID = PROJECT_ID # @param {type:"string"}
REGION = "us-central1"

vertexai.init(project={PROJECT_ID}, location=REGION)

In [ ]:
import json
import textwrap

# Utils
import time
import uuid
from typing import List

import numpy as np
import vertexai

# Vertex AI
from google.cloud import aiplatform

print(f"Vertex AI SDK version: {aiplatform.__version__}")

# LangChain
import langchain

print(f"LangChain version: {langchain.__version__}")

from langchain.chains import RetrievalQA
from langchain.document_loaders import GCSDirectoryLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Import custom Matching Engine packages
# from utils.matching_engine import MatchingEngine
# from utils.matching_engine_utils import MatchingEngineUtils
# Import custom Matching Engine packages
from langchain_google_vertexai import VertexAI , VertexAIEmbeddings , VectorSearchVectorStore

Vertex AI SDK version: 1.47.0
LangChain version: 0.1.14


# connecting to bigquery to extract the text data

In [ ]:
# load the BQ Table into a Pandas Dataframe
import pandas as pd
from google.cloud import bigquery


bq_client = bigquery.Client(project=PROJECT_ID)
QUERY_TEMPLATE = """
        SELECT * FROM `....Pubmed.pubmed` where content != '';
        """
# query_params=[
#         bigquery.ArrayQueryParameter("q1","DATE", q1),
#         bigquery.ArrayQueryParameter("q2","DATE", q2),
#         bigquery.ArrayQueryParameter("q3","DATE", q3),
#         bigquery.ArrayQueryParameter("q4","DATE", q4),
#         bigquery.ArrayQueryParameter("rule_name","STRING", rule_name),
#         bigquery.ArrayQueryParameter("insert_timestamp","DATE", insert_timestamp),
#         bigquery.ArrayQueryParameter("Manufacturer","STRING", Manufacturer),
#         bigquery.ArrayQueryParameter("partner_code","STRING", partner_code),
    # ]
try:
  pubmed = bq_client.query(QUERY_TEMPLATE)  # Make an API request.
  pubmed_data = pubmed.to_dataframe()
except Exception as e:
  print('Error',e,'Data_not_found')

# examine the data
pubmed_data.head()

,Title,content
0,"Impact of Alcoholism – Kerala.pdf,page:66",(17.9%)for the Adolescents. A great majority (...
1,"Gururaj_-Alcohol-report-NIMHANS.pdf,page:80","(Carey et al, 2003b). Among psychiatric inpati..."
2,"Gururaj_-Alcohol-report-NIMHANS.pdf,page:116",(although 33.3% had current alcohol use and 16...
3,"Gururaj_-Alcohol-report-NIMHANS.pdf,page:122",(d) Public health should be given proper defer...
4,Combined_Pharmacotherapies_and_Behavioral_Inte...,"(inthiscase,approximately1inevery report.Poten..."


# Load the text embeddings model
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

# Load the text embeddings model

In [ ]:

from langchain_google_vertexai import VertexAIEmbeddings

model = VertexAIEmbeddings(model_name="textembedding-gecko@003")

# Load the biquery where data for llm is stored

In [ ]:
# from langchain.document_loaders import BigQueryLoader #Class for storing a piece of text and associated metadata.
# BASE_QUERY = """SELECT * FROM `......Pubmed.pubmed` where content != '';"""
# loader = BigQueryLoader(BASE_QUERY,project="google.com:centering-river-118922")
# documents = loader.load()

/root/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `BigQueryLoader` was deprecated in LangChain 0.0.32 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import BigQueryLoader`.
  warn_deprecated(


In [ ]:
# Add document metadata to all the document of documents and formatting page_content to only contain contents of BQ table pubmed as it contained both title and content
# for document in documents:
#   document.metadata={'source':document.page_content.split('\n')[0]}
#   document.page_content=document.page_content.split('\n')[1]


In [ ]:
# documents[0].metadata

{'source': 'Title: Impact of Alcoholism – Kerala.pdf,page:66'}

# Chunk documents
Split the documents to smaller chunks. When splitting the document, ensure a few chunks can fit within the context length of LLM.

In [ ]:
# # split the documents into chunks
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=50,
#     separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
# )
# doc_splits = text_splitter.split_documents(documents)

# # Add chunk number to metadata
# for idx, split in enumerate(doc_splits):
#     split.metadata["chunk"] = idx

# print(f"# of documents = {len(doc_splits)}")

# of documents = 4405


# Create Matching Engine Index and Endpoint for Retrieval


In [ ]:
PROJECT_ID =..  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
ME_REGION = "<region>"
ME_INDEX_NAME = f"..."  # @param {type:"string"}
ME_EMBEDDING_DIR = f".."  # @param {type:"string"}
ME_DIMENSIONS = 768  # when using Vertex PaLM Embedding

In [ ]:
ME_EMBEDDING_DIR

# Make a Google Cloud Storage bucket for your Matching Engine index

In [ ]:
# ! set -x && gsutil mb -p $PROJECT_ID -l us-central1 gs://"........"

# Create Index
You can create index on Vertex AI Matching Engine for batch updates or streaming updates.

This notebook creates Matching Engine Index:

With streaming updates
With default configuration - e.g. small shard size
You can update the index configuration in the Matching Engine utilities script.



While the index is being created and deployed, you can read more about Matching Engine's ANN service which uses a new type of vector quantization developed by Google Research: Accelerating Large-Scale Inference with Anisotropic Vector Quantization.

For more information about how this works, see Announcing ScaNN: Efficient Vector Similarity Search.

In [ ]:
# # NOTE : This operation can take upto 30 seconds
# my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
#     display_name="langchain-index",
#     dimensions=768,
#     approximate_neighbors_count=150,
#     distance_measure_type="DOT_PRODUCT_DISTANCE",
#     index_update_method="STREAM_UPDATE",  # allowed values BATCH_UPDATE , STREAM_UPDATE
# )
# if my_index:
#     print(my_index.name)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/223914638453/locations/us-central1/indexes/5634236230305579008/operations/5421764988446441472
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/223914638453/locations/us-central1/indexes/5634236230305579008
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/223914638453/locations/us-central1/indexes/5634236230305579008')


5634236230305579008


# Deploy Index to Endpoint
Deploy index to Index Endpoint on Matching Engine. This notebook deploys the index to a public endpoint. The deployment operation creates a public endpoint that will be used for querying the index for approximate nearest neighbors.

For deploying index to a Private Endpoint, refer to the documentation to set up pre-requisites.

In [ ]:
# # Create an endpoint
# index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
#     display_name=f"langchain-index-endpoint", public_endpoint_enabled=True
# )
# if index_endpoint:
#     print(f"Index endpoint resource name: {index_endpoint.name}")
#     print(
#         f"Index endpoint public domain name: {index_endpoint.public_endpoint_domain_name}"
#     )

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngineIndexEndpoint backing LRO: projects/223914638453/locations/us-central1/indexEndpoints/2919832292396367872/operations/5487911607973445632
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint created. Resource name: projects/223914638453/locations/us-central1/indexEndpoints/2919832292396367872
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:To use this MatchingEngineIndexEndpoint in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/223914638453/locations/us-central1/indexEndpoints/2919832292396367872')


Index endpoint resource name: 2919832292396367872
Index endpoint public domain name: 1238255132.us-central1-223914638453.vdb.vertexai.goog


In [ ]:
# #Deploy Index to endpoint
# # NOTE : This operation can take upto 20 minutes
# my_index_endpoint = index_endpoint.deploy_index(
#     index=my_index, deployed_index_id="langchain_index_endpoint_deployed_index"
# )

# my_index_endpoint.deployed_indexes

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/223914638453/locations/us-central1/indexEndpoints/2919832292396367872
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/223914638453/locations/us-central1/indexEndpoints/2919832292396367872/operations/9132731081399730176
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/223914638453/locations/us-central1/indexEndpoints/2919832292396367872


[id: "langchain_index_endpoint_deployed_index"
index: "projects/223914638453/locations/us-central1/indexes/5634236230305579008"
create_time {
  seconds: 1713616537
  nanos: 759710000
}
index_sync_time {
  seconds: 1713616537
  nanos: 759710000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

# Initialize LangChain Models
You initialize LangChain Models with the pre-trained text, chat and embeddings generation model called text-bison@001, chat-bison@001 and textembedding-gecko@001 respectively.

In [ ]:
# Text model instance integrated with langChain
llm = VertexAI(
    model_name="text-bison@002",
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Embeddings API integrated with langChain
embeddings = VertexAIEmbeddings(
    model_name= "textembedding-gecko@003"
)

In [ ]:
# my_index.name='5634236230305579008'

'3387502966201122816'

In [ ]:
#my_index.name

'5634236230305579008'

In [ ]:
#index_endpoint.name

'2919832292396367872'

In [ ]:
# index_endpoint.name='2919832292396367872'

'4871579780908056576'

In [ ]:
#gcs_bucket_name='centering-river-118922-me-bucket'

'centering-river-118922-me-bucket'

# Configure Matching Engine as Vector Store
Initialize Matching Engine vector store with text embeddings model

In [ ]:
# initialize vector store
me = VectorSearchVectorStore.from_components(
    project_id=PROJECT_ID,
    region='us-central1',
    gcs_bucket_name=f"gs://{ME_EMBEDDING_DIR}".split("/")[2],
    embedding=embeddings,
    index_id='5634236230305579008',
    endpoint_id='2919832292396367872',
    stream_update=True,
)

# Add documents as embeddings in Matching Engine as index
The document chunks are transformed as embeddings (vectors) using Vertex AI Embeddings API and added to the index with streaming index update. With Streaming Updates, you can update and query your index within a few seconds.

The original document text is stored on Cloud Storage bucket had referenced by id.

Prepare text and metadata to be added to the vectors

In [ ]:
# # Store docs as embeddings in Matching Engine index
# # It may take a while since API is rate limited
# texts = [doc.page_content for doc in doc_splits]
# metadatas = [doc.metadata for doc in doc_splits]

In [ ]:
# print(len(texts))

4405


In [ ]:
# print(len(texts[0]))

706


In [ ]:
# for i in range (0 , len(texts)):
#   if len(texts[i])>20000:
#     print(texts[i])
#     print(metadatas[i])


# Add embeddings to the vector store

NOTE: Depending on the volume and size of documents, this step may take time.

In [ ]:
# # This will add embedding to vector engine and it's respective text to GCS
# for i in range (0 , len(texts) , 100):
#     me.add_texts(texts[i:i+100], metadatas[i:i+100])

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Upserting datapoints MatchingEngineIndex index: projects/223914638453/locations/us-central1/indexes/5634236230305579008
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex index Upserted datapoints. Resource name: projects/223914638453/locations/us-central1/indexes/5634236230305579008
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Upserting datapoints MatchingEngineIndex index: projects/223914638453/locations/us-central1/indexes/5634236230305579008
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex index Upserted datapoints. Resource name: projects/223914638453/locations/us-central1/indexes/5634236230305579008
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Upserting datapoints MatchingEngineIndex index: projects/223914638453/locations/us-central1/indexes/5634236230305579008
INFO:google.cloud.aiplatform.matching

In [ ]:
# # # This will add embedding to vector engine and it's respective text to GCS
# # for i in range (0 , len(texts) , 500):
# #     me.add_texts(texts[i:i+500], metadatas[i:i+500])
# INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Upserting datapoints MatchingEngineIndex index: projects/223914638453/locations/us-central1/indexes/5634236230305579008
# INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex index Upserted datapoints. Resource name: projects/223914638453/locations/us-central1/indexes/5634236230305579008
# INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Upserting datapoints MatchingEngineIndex index: projects/223914638453/locations/us-central1/indexes/5634236230305579008
# INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex index Upserted datapoints. Resource name: projects/223914638453/locations/us-central1/indexes/5634236230305579008
# INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Upserting datapoints MatchingEngineIndex index: projects/223914638453/locations/us-central1/indexes/5634236230305579008
# INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex index Upserted datapoints. Resource name: projects/223914638453/locations/us-central1/indexes/5634236230305579008
# ---------------------------------------------------------------------------
# _InactiveRpcError                         Traceback (most recent call last)
# /usr/local/lib/python3.10/dist-packages/google/api_core/grpc_helpers.py in error_remapped_callable(*args, **kwargs)
#      71         try:
# ---> 72             return callable_(*args, **kwargs)
#      73         except grpc.RpcError as exc:

# 21 frames
# _InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
# 	status = StatusCode.INVALID_ARGUMENT
# 	details = "Unable to submit request because the input token count is 20926 but the model supports up to 20000. Reduce the input token count and try again. You can also use the CountTokens API to calculate prompt token count and billable characters. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models"
# 	debug_error_string = "UNKNOWN:Error received from peer ipv4:108.177.11.95:443 {grpc_message:"Unable to submit request because the input token count is 20926 but the model supports up to 20000. Reduce the input token count and try again. You can also use the CountTokens API to calculate prompt token count and billable characters. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models", grpc_status:3, created_time:"2024-04-20T13:14:06.404622008+00:00"}"
# >

# The above exception was the direct cause of the following exception:

# InvalidArgument                           Traceback (most recent call last)
# /usr/local/lib/python3.10/dist-packages/google/api_core/grpc_helpers.py in error_remapped_callable(*args, **kwargs)
#      72             return callable_(*args, **kwargs)
#      73         except grpc.RpcError as exc:
# ---> 74             raise exceptions.from_grpc_error(exc) from exc
#      75
#      76     return error_remapped_callable

# InvalidArgument: 400 Unable to submit request because the input token count is 20926 but the model supports up to 20000. Reduce the input token count and try again. You can also use the CountTokens API to calculate prompt token count and billable characters. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models

In [ ]:
me.similarity_search("prolong alcohol intake impact human health?", k=5)

[Document(page_content='content: Open Access Review Article DOI: 10.7759/cureus.30057 Effects of Alcohol Consumption on Various Systems of the Human Body: A Systematic Review Jerin Varghese 1 , Sarika Dakhode 2 Received 09/16/2022 Review began 09/20/2022 Review ended 09/30/2022 1. Medical School, Jawaharlal Nehru Medical College, Datta Meghe Institute of Medical Sciences, Wardha, IND 2. Published 10/08/2022 Community Medicine, Jawaharlal Nehru Medical College, Datta Meghe Institute of Medical Sciences, Wardha, IND Varghese et al. This is an open access Corresponding author: Jerin Varghese, jerinazero@gmail.com article distributed under the terms of the Creative Commons Attribution License CC- BY 4.0., which permits unrestricted use, distribution, and reproduction in any Abstract medium, provided the original author and source are credited. Prolonged alcohol intake for many years has been known to cause serious ailments in human beings since time memorial', metadata={'source': 'Title: E

In [ ]:
me.asimilarity_search_with_score("prolong alcohol intake impact human health?", k=5)

<coroutine object VectorStore.asimilarity_search_with_score at 0x7c1951ee3060>

In [ ]:
me.similarity_search("What is e-cigarrette?", k=5)

[Document(page_content='content: Electronic cigarettes (e-cigarettes) are devices that allow the user to inhale nicotine in a vapor, and are primarily marketed as a means of quitting smoking and a less harmful replacement for traditional cigarette smoking. However, further research is needed to determine if vaping nicotine via e-cigarettes can be effective. Conversely, nicotine has been considered a gateway drug to alcohol and other addictive drugs and e-cigarettes containing nicotine may have the same effects. Previous reports have shown that e-cigarette use may open the gate for the use of other drugs including conventional cigarettes, cannabis, opioids, etc. The increasing prevalence of e-cigarettes, particularly among youth and adolescents in the last decade have led to an increase in the dual use of e-cigarettes with alcohol, cannabis, and other illicit drug use like heroin and 3-4-methylenedioxymethamphetamine (MDMA)', metadata={'source': 'Title: E-cigarettes may serve as a gatew

In [ ]:
# Create chain to answer questions
NUMBER_OF_RESULTS = 5
SEARCH_DISTANCE_THRESHOLD = 0.8

# Expose index to the retriever
retriever = me.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": NUMBER_OF_RESULTS,
        "search_distance": SEARCH_DISTANCE_THRESHOLD,
    },
    filters=None,
)

In [ ]:
template = """SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: {question}

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

=============
{context}
=============

Question: {question}
Helpful Answer:"""

In [ ]:
# Uses LLM to synthesize results from the search index.
# Use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    },
)

In [ ]:
# Enable for troubleshooting
qa.combine_documents_chain.verbose = True
qa.combine_documents_chain.llm_chain.verbose = True
qa.combine_documents_chain.llm_chain.llm.verbose = True

In [ ]:
def formatter(result):
    print(f"Query: {result['query']}")
    print("." * 80)
    if "source_documents" in result.keys():
        for idx, ref in enumerate(result["source_documents"]):
            print("-" * 80)
            print(f"REFERENCE #{idx}")
            print("-" * 80)
            if "score" in ref.metadata:
                print(f"Matching Score: {ref.metadata['score']}")
            if "source" in ref.metadata:
                print(f"Document Source: {ref.metadata['source']}")
            if "document_name" in ref.metadata:
                print(f"Document Name: {ref.metadata['document_name']}")
            print("." * 80)
            print(f"Content: \n{wrap(ref.page_content)}")
    print("." * 80)
    print(f"Response: {wrap(result['result'])}")
    print("." * 80)


def wrap(s):
    return "\n".join(textwrap.wrap(s, width=120, break_long_words=False))


def ask(
    query,
    qa=qa,
    k=NUMBER_OF_RESULTS,
    search_distance=SEARCH_DISTANCE_THRESHOLD,
    filters={},
):
    qa.retriever.search_kwargs["search_distance"] = search_distance
    qa.retriever.search_kwargs["k"] = k
    qa.retriever.search_kwargs["filters"] = filters
    result = qa({"query": query})
    return formatter(result)

In [ ]:
ask("prolong alcohol intake impact human health?")

/root/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: prolong alcohol intake impact human health?

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

content: Open Access Review Article DOI: 10.7759/cureus.30057 Effects of Alcohol Consumption on Various Systems of the Human Body: A Systematic Review Jerin Varghese 1 , Sarika Dakhode 2 Received 09/16/2022 Review began 09/20/2022 Review ended 09/30/2022 1. Medical School, Jawaharlal Nehru Medical College, Datta Meghe Institute of Medical Science

In [ ]:
ask("Why are causal associations?")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: Why are causal associations?

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

. First, most human studies are correlative in nature, meaning that causal associations cannot be made between in uterocannabis exposure and health and behavioral outcomes later in life. Due to obvious ethical constraints, it is not possible to randomly assign pregnant mothers to cannabis or other drug exposure conditions-a requirement needed to establish causa

In [ ]:
ask("What is Electronic cigarettes?")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: What is Electronic cigarettes?

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

content: Electronic cigarettes (e-cigarettes) are devices that allow the user to inhale nicotine in a vapor, and are primarily marketed as a means of quitting smoking and a less harmful replacement for traditional cigarette smoking. However, further research is needed to determine if vaping nicotine via e-cigarettes can be effective. Conversely, nicotine has 

# Trying gemini model

In [ ]:
llm1 = VertexAI(model_name="gemini-1.0-pro",max_output_tokens=8042, temperature=0)

In [ ]:
template = """SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: {question}

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

=============
{context}
=============

Question: {question}
Helpful Answer:"""

In [ ]:
# Uses LLM to synthesize results from the search index.
# Use Vertex PaLM Text API for LLM
qa1 = RetrievalQA.from_chain_type(
    llm=llm1,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    },
)

In [ ]:
# Enable for troubleshooting
qa1.combine_documents_chain.verbose = True
qa1.combine_documents_chain.llm_chain.verbose = True
qa1.combine_documents_chain.llm_chain.llm.verbose = True

In [ ]:
def formatter(result):
    print(f"Query: {result['query']}")
    print("." * 80)
    if "source_documents" in result.keys():
        for idx, ref in enumerate(result["source_documents"]):
            print("-" * 80)
            print(f"REFERENCE #{idx}")
            print("-" * 80)
            if "score" in ref.metadata:
                print(f"Matching Score: {ref.metadata['score']}")
            if "source" in ref.metadata:
                print(f"Document Source: {ref.metadata['source']}")
            if "document_name" in ref.metadata:
                print(f"Document Name: {ref.metadata['document_name']}")
            print("." * 80)
            print(f"Content: \n{wrap(ref.page_content)}")
    print("." * 80)
    print(f"Response: {wrap(result['result'])}")
    print("." * 80)


def wrap(s):
    return "\n".join(textwrap.wrap(s, width=120, break_long_words=False))


def ask(
    query,
    qa=qa1,
    k=NUMBER_OF_RESULTS,
    search_distance=SEARCH_DISTANCE_THRESHOLD,
    filters={},
):
    qa1.retriever.search_kwargs["search_distance"] = search_distance
    qa1.retriever.search_kwargs["k"] = k
    qa1.retriever.search_kwargs["filters"] = filters
    result = qa1({"query": query})
    return formatter(result)

In [ ]:
ask("prolong alcohol intake impact human health?")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: prolong alcohol intake impact human health?

I'm an adult. Provide a detailed answer using bullet points.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

content: Open Access Review Article DOI: 10.7759/cureus.30057 Effects of Alcohol Consumption on Various Systems of the Human Body: A Systematic Review Jerin Varghese 1 , Sarika Dakhode 2 Received 09/16/2022 Review began 09/20/2022 Review ended 09/30/2022 1. Medical School, Jawaharlal Nehru Medical College, Datta Meghe Institute of Medical Sciences, Wardha, IND 2. Published 10/08/2022 Community Medicine, J

In [ ]:
llm1 = VertexAI(model_name="gemini-1.0-pro",max_output_tokens=8042, temperature=0.9)

In [ ]:

template = """SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: {question}

I'm an adult. Provide a detailed answer using bullet points.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

=============
{context}
=============

Question: {question}
Helpful Answer:"""

In [ ]:
# Use Vertex PaLM Text API for LLM
qa1 = RetrievalQA.from_chain_type(
    llm=llm1,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    },
)

In [ ]:
# Enable for troubleshooting
qa1.combine_documents_chain.verbose = True
qa1.combine_documents_chain.llm_chain.verbose = True
qa1.combine_documents_chain.llm_chain.llm.verbose = True

In [ ]:
def formatter(result):
    print(f"Query: {result['query']}")
    print("." * 80)
    if "source_documents" in result.keys():
        for idx, ref in enumerate(result["source_documents"]):
            print("-" * 80)
            print(f"REFERENCE #{idx}")
            print("-" * 80)
            if "score" in ref.metadata:
                print(f"Matching Score: {ref.metadata['score']}")
            if "source" in ref.metadata:
                print(f"Document Source: {ref.metadata['source']}")
            if "document_name" in ref.metadata:
                print(f"Document Name: {ref.metadata['document_name']}")
            print("." * 80)
            print(f"Content: \n{wrap(ref.page_content)}")
    print("." * 80)
    print(f"Response: {wrap(result['result'])}")
    print("." * 80)


def wrap(s):
    return "\n".join(textwrap.wrap(s, width=120, break_long_words=False))


def ask(
    query,
    qa=qa1,
    k=NUMBER_OF_RESULTS,
    search_distance=SEARCH_DISTANCE_THRESHOLD,
    filters={},
):
    qa1.retriever.search_kwargs["search_distance"] = search_distance
    qa1.retriever.search_kwargs["k"] = k
    qa1.retriever.search_kwargs["filters"] = filters
    result = qa1({"query": query})
    return formatter(result),result

In [ ]:
result,result1=ask("prolong alcohol intake impact human health?")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: prolong alcohol intake impact human health?

I'm an adult. Provide a detailed answer using bullet points.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

content: Open Access Review Article DOI: 10.7759/cureus.30057 Effects of Alcohol Consumption on Various Systems of the Human Body: A Systematic Review Jerin Varghese 1 , Sarika Dakhode 2 Received 09/16/2022 Review began 09/20/2022 Review ended 09/30/2022 1. Medical School, Jawaharlal Nehru Medical College, Datta Meghe Institute of Medical Sciences, Wardha, IND 2. Published 10/08/2022 Community Medicine, J

# Topic Modelling

In [ ]:
from vertexai.generative_models import GenerationConfig, GenerativeModel

In [ ]:
model = GenerativeModel("gemini-1.0-pro")

In [ ]:
text=result1.get('result')

In [ ]:
modified_prompt =  """
Encoded Text Representation: {}

Generate 5 questions... """.format(text)

In [ ]:
generation_config = GenerationConfig(
    temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8
)

print(model.generate_content(modified_prompt, generation_config=generation_config).text)

## 5 Questions about Prolonged Alcohol Intake and Its Effects on Human Health:

1. **What are the specific mechanisms by which prolonged alcohol intake leads to brain damage and cognitive decline?**
2. **How does the severity of negative health impacts vary depending on the amount and duration of alcohol consumption?**
3. **Is there a safe limit for alcohol consumption that minimizes health risks while still offering potential benefits?**
4. **What are the most effective strategies for preventing and treating alcohol-related health problems?**
5. **How can we better educate individuals and communities about the risks and potential benefits of alcohol consumption?**


# Interview question generation

In [ ]:
prompt = """
Encoded Text Representation: {}
Generate 10 questions that test student knowledge about the topic.


""".format(text)

In [ ]:
print(model.generate_content(prompt, generation_config=generation_config).text)

## 10 Questions to Test Student Knowledge about Prolonged Alcohol Intake and Its Effects on Human Health:

**Multiple Choice:**

1. Which of the following is NOT a negative impact of prolonged alcohol intake on the brain?
    a) Reduced grey and white matter volume
    b) Improved cognitive function
    c) Increased risk of dementia
    d) Higher vulnerability in adolescents

2. Which of the following is the most common type of liver damage caused by prolonged alcohol intake?
    a) Fatty liver disease
    b) Alcoholic hepatitis
    c) Cirrhosis
    d) Liver cancer

3. Which of the following is NOT a cardiovascular complication associated with prolonged alcohol intake?
    a) High blood pressure
    b) Cardiomyopathy
    c) Osteoporosis
    d) Arrhythmia

4. Which of the following cancers is NOT associated with increased risk due to prolonged alcohol intake?
    a) Mouth cancer
    b) Lung cancer
    c) Esophageal cancer
    d) Breast cancer

5. Which of the following mental health con

In [ ]:
print('work in progress for business use case')

work in progress for business use case
